<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-7---Continuous-time-averaging-systems" data-toc-modified-id="Chapter-7---Continuous-time-averaging-systems-1">Chapter 7 - Continuous-time averaging systems</a></span><ul class="toc-item"><li><span><a href="#7.1-Example-Systems" data-toc-modified-id="7.1-Example-Systems-1.1">7.1 Example Systems</a></span><ul class="toc-item"><li><span><a href="#7.1.1-Example-#1:-Continuous-time-opinion-dynamics" data-toc-modified-id="7.1.1-Example-#1:-Continuous-time-opinion-dynamics-1.1.1">7.1.1 Example #1: Continuous-time opinion dynamics</a></span></li><li><span><a href="#7.1.3-Example-#3:-Discretization-of-partial-differential-equations" data-toc-modified-id="7.1.3-Example-#3:-Discretization-of-partial-differential-equations-1.1.2">7.1.3 Example #3: Discretization of partial differential equations</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
%matplotlib widget

# Import packages
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# For interactive graphs
import ipywidgets as widgets

# Import self defined functions
import lib  # General library

# Settings
custom_figsize= (6, 4) # Might need to change this value to fit the figures to your screen
custom_figsize_square = (5, 5) 

# Chapter 7 - Continuous-time averaging systems

These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** In this script it is necessary to execute cell by cell chronologically due to reocurring examples (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

## 7.1 Example Systems

In this section we repeat some example covered in the book for visualization and experimentation. Since we use continuous time models, the simulation must be done in some way of discretized manner. These are shown below.

### 7.1.1 Example #1: Continuous-time opinion dynamics

For a simulating example of a French-Harary-DeGroot discrete-time averaging model to a Abelson's continuous timeopinion dynamics model, we actually take a look at **Exercise 5.1** and simulate the results here. The simulation of the continuous-time model is done with $x(t+\tau) = x(t) + -\bar{L} x(t) \tau$.

First we define our network, plot the graph and as for Exercise 5.1 (i), we draw the condensation graph.

In [ ]:
# Adjacency Matrix
A_711 = np.array([
    [0.15, 0.15, 0.1, 0.2, 0.4],
    [0, 0.55, 0, 0, 0.45],
    [0.3, 0.05, 0.05, 0, 0.6],
    [0, 0.4, 0.1, 0.5, 0],
    [0, 0.3, 0, 0, 0.7]
])

# Giving pre defined positions away
pos711 = {0:[0.1,0.2],1:[.4,.5],2:[.5,.2],3:[.8,.5], 4:[.9,.2]}

G_711 = lib.create_G_from_adj_matrix(A_711)

fig, axs711 = plt.subplots(1, 3, figsize=(custom_figsize[0]*1.2, custom_figsize[1]))

lib.plot_condensated_graph(G_711, axs711, pos711)

The next step is to actually simulate the system and plot the results. We can play with the simulation parameters and realize, that as expected the same result as for discretized and continuous system are achieved.

In [ ]:
# Settings, can change these
tau = 0.2
t_total = 200

x_init = np.diag(A_711)  # Initial condition

# Laplace
L_711 = np.diag(np.sum(A_711, 1)) - A_711

# Simulation
states = np.zeros((t_total,L_711.shape[0]))
states[0,:]=x_init
for i in range(1,t_total):
    states[i,:] = states[i-1,:] + -L_711 @ states[i-1,:] * tau

# From here, interactive network representation
fig, ax711 = plt.subplots(figsize=custom_figsize)
fig, v_bound, pos = lib.init_network_sim_plot(G_711, states, fig, pos=pos711)

def inter(timestep):
    lib.update_network(timestep['new'], G=G_711, states_m=states, ax=ax711, vbound=v_bound, pos=pos711)
    return None

# Plot initial configuration
lib.update_network(0, G=G_711, states_m=states, ax=ax711, vbound=v_bound, pos=pos711)


# Widget
# If this cell is executed twice we are making sure in the following, that the previous widget instances are all closed
try:
    [c.close() for c in widget711.children]  # Note: close_all() does also affect plot, thus list compr.
except NameError:  # Only want to except not defined variable error
    pass

widget711 = lib.create_widgets_play_slider(fnc=inter, minv=0, maxv=t_total-1, step=1, play_speed=1000)

display(widget711)

### 7.1.3 Example #3: Discretization of partial differential equations

For some intuition and exploration, the heat equation presented in the book is presented below in a minimum example.

In [ ]:
# Settings for grid
n = 4

# Simulation Parameter can change these
tau2 = 0.01
t_total2 = 200
c = 0.1  # thermal diffusivity
h = 0.2  # Grid length

# Initial
x_init2 = np.random.rand(n**2) * 100

# Two dimensional grid graph
G_grid = nx.generators.lattice.grid_graph([n, n])
# Save Positions for plot later and normalize coordinates from original node coordinate
pos712 = dict(zip(range(0, n**2), [tuple([y/n for y in x]) for x in G_grid.nodes]))

# Relabel for simplicity
G_grid = nx.relabel.relabel_nodes(G_grid, dict(zip(G_grid.nodes, range(0, n**2))))

A_grid = nx.linalg.graphmatrix.adjacency_matrix(G_grid, nodelist=G_grid.nodes).toarray()
L_grid = np.diag(np.sum(A_grid, 1)) - A_grid

# Simulation
states2 = np.zeros((t_total2,L_grid.shape[0]))
states2[0,:] = x_init2
for i in range(1,t_total2):
    states2[i,:] = states2[i-1,:] + -(c/h**2)*L_grid @ states2[i-1,:] * tau2

In [ ]:
# From here, interactive network representation
fig, ax712 = plt.subplots(figsize=custom_figsize)
fig, v_bound, pos712 = lib.init_network_sim_plot(G_grid, states2, fig, pos=pos712)

def inter2(timestep):
    lib.update_network(timestep['new'], G=G_grid, states_m=states2, ax=ax712, vbound=v_bound, pos=pos712)
    return None

# Plot initial configuration
lib.update_network(0, G=G_grid, states_m=states2, ax=ax712, vbound=v_bound, pos=pos712)


# Widget
# If this cell is executed twice we are making sure in the following, that the previous widget instances are all closed
try:
    [c.close() for c in widget712.children]  # Note: close_all() does also affect plot, thus list compr.
except NameError:  # Only want to except not defined variable error
    pass

widget712 = lib.create_widgets_play_slider(fnc=inter2, minv=0, maxv=t_total2-1, step=1, play_speed=1000)

display(widget712)